# Simple Implimentation of RNN from scratch 

In [4]:
import numpy as np

# Defining activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1 - np.tanh(x)**2

# Defining the RNN class
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = learning_rate

        # Initializing weights
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01  # input -> hidden
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden -> hidden
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # hidden -> output

        # Initializing biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        """
        Forward pass
        inputs: list of input vectors (each input_size x 1)
        """
        self.last_inputs = inputs
        self.last_hs = {0: np.zeros((self.hidden_size, 1))}  # initial hidden state

        for t, x in enumerate(inputs):
            prev_h = self.last_hs[t]
            h = tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, prev_h) + self.bh)
            self.last_hs[t + 1] = h

        # Final output
        y = np.dot(self.Why, self.last_hs[len(inputs)]) + self.by
        return y

    def backward(self, d_y, learning=True):
        """
        Backward pass
        d_y: gradient on output
        """
        n = len(self.last_inputs)
        
        # Gradients initialization
        dWhy = np.dot(d_y, self.last_hs[n].T)
        dby = d_y

        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dbh = np.zeros_like(self.bh)

        dh_next = np.zeros((self.hidden_size, 1))

    
        for t in reversed(range(n)):
            h = self.last_hs[t+1]
            prev_h = self.last_hs[t]

            dh = np.dot(self.Why.T, d_y) + dh_next  # backprop into h
            dh_raw = dtanh(h) * dh

            dbh += dh_raw
            dWxh += np.dot(dh_raw, self.last_inputs[t].T)
            dWhh += np.dot(dh_raw, prev_h.T)

            dh_next = np.dot(self.Whh.T, dh_raw)

    
        if learning:
            for param, dparam in zip(
                [self.Wxh, self.Whh, self.Why, self.bh, self.by],
                [dWxh, dWhh, dWhy, dbh, dby]
            ):
                param -= self.lr * dparam

    def train(self, inputs, target):
        """
        inputs: list of inputs
        target: target output
        """
        output = self.forward(inputs)
        loss = np.square(output - target).sum()  # simple MSE loss
        d_y = 2 * (output - target)  # derivative of loss w.r.t output
        self.backward(d_y)
        return loss

    def predict(self, inputs):
        output = self.forward(inputs)
        return output


In [5]:
# Creating the RNN
rnn = SimpleRNN(input_size=1, hidden_size=10, output_size=1, learning_rate=0.01)

# Training data: sequence -> sum
dataset = [
    ([np.array([[1]]), np.array([[2]])], np.array([[3]])),  # 1+2=3
    ([np.array([[0]]), np.array([[5]])], np.array([[5]])),  # 0+5=5
    ([np.array([[3]]), np.array([[4]])], np.array([[7]])),  # 3+4=7
    ([np.array([[2]]), np.array([[1]])], np.array([[3]]))   # 2+1=3
]

# Training
for epoch in range(1000):
    total_loss = 0
    for inputs, target in dataset:
        loss = rnn.train(inputs, target)
        total_loss += loss
    if epoch % 100 == 0:
        print(f"Epoch {epoch} Loss: {total_loss:.4f}")

# Testing
test_input = [np.array([[2]]), np.array([[3]])]  # Should predict 5
pred = rnn.predict(test_input)
print("Prediction for 2+3:", pred.flatten()[0])


Epoch 0 Loss: 87.4627
Epoch 100 Loss: 3.3846
Epoch 200 Loss: 8.2014
Epoch 300 Loss: 4.7297
Epoch 400 Loss: 10.0501
Epoch 500 Loss: 0.2479
Epoch 600 Loss: 0.0001
Epoch 700 Loss: 0.0000
Epoch 800 Loss: 0.0000
Epoch 900 Loss: 0.0000
Prediction for 2+3: 6.379563412931214
